Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [1]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
from sklearn.model_selection import train_test_split

In [2]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [4]:
X_train = X_train.head()
y_train = y_train.head()

In [5]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

def leakyReLU(x):
    return max(0.1*x, x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic
    lRelu = leakyReLU

In [17]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu, 32))
network.add(layer(ActFunc.hb, 64))
network.add(layer(ActFunc.log, 128))
network.add(layer(ActFunc.relu, 1))
# network.add(layer(ActFunc.relu, 32))
# network.add(layer(ActFunc.relu, 64))
# network.add(layer(ActFunc.relu, 128))
# network.add(layer(ActFunc.relu, 1))

Applying NN on the concrete data

In [19]:
# PSO test!
import pso

swarmsize = 100
alpha = 0.7
beta = 2
gamma = 1.5
delta = 1.5
epsilon = 0.4
n_iter = 100

particle_swarm_opti = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter, prints=True)
opti_particle, best_mae_arr = particle_swarm_opti.optimise()

Current best mae: 34.338
Current best mae: 32.7529694210205
Current best mae: 28.612568981906065
Current best mae: 27.971339540736686
Current best mae: 26.644353748823317
Current best mae: 26.549146224206844
Current best mae: 24.641139946663824
Current best mae: 23.402834435563943
Current best mae: 22.912784355571638
Current best mae: 22.791003321268768
Current best mae: 21.756439969178942
Current best mae: 21.56394635096
Current best mae: 20.54023895161038
Current best mae: 19.160247492245805
Current best mae: 16.45716552076347
Current best mae: 14.64474808206337
Current best mae: 14.628386969976415
Current best mae: 14.29883888091588
Current best mae: 12.123206672541349
Current best mae: 11.722661961106859
Current best mae: 10.508106219619942
Current best mae: 10.264336800470964
Current best mae: 10.224454858931058
Current best mae: 10.009864274782885
Current best mae: 9.682997550201215
Current best mae: 9.466421956650247
Current best mae: 9.203946931356082
Current best mae: 9.182852

In [20]:
# Potential ranges of PSO parameters to try
# Could try them like a gridsearch but its probably a bit too much for that, tuning one at a time is probably the way to go

alphaRange = np.arange(0.4,0.9,0.1)
betaRange = np.arange(1.5,2.5,0.1)
gammaRange = np.arange(1.5,2.5,0.1)
deltaRange = np.arange(1.5,2.5,0.1)
epsilonRange = np.arange(0.1,0.5,0.1) # not sure about the range of the learning rate, might need to experiment or look for more sources

In [9]:
#alpha tuning

accuracies = []

for alpha in alphaRange:
    network = neuralNet()

In [10]:
# Try neural network architectures
# 2-10 layers
# 3-10 neurons per layer

layers = range(1, 5) # 1 to x hidden layer
#neurons = range(2, 30) # 2 to 30 neurons
# Try different layers
# Constant 4 neurons per layer, activation func relu

accuraciesForLayers = []
for numLayers in layers:
    network = neuralNet()
    network.add(layer(ActFunc.hb,4))
    for i in range(numLayers - 1):
        network.add(layer(ActFunc.relu,4))
    network.add(layer(ActFunc.relu, 1)) # output layer
    layerTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = layerTestPSO.optimise()
    accuraciesForLayers.append(best_mae_arr[-1])

In [11]:
accuraciesForLayers

[6.555264950844917, 8.99807228712115, 7.945903903327652, 9.003288963387677]

In [12]:
neurons = range(3,11)

accuraciesForNeuronCounts = []

for neuronCount in neurons:
    network = neuralNet()
    network.add(layer(ActFunc.hb,neuronCount))
    for i in range(4):
        network.add(layer(ActFunc.relu,neuronCount))
    network.add(layer(ActFunc.relu,1))
    neuronsTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = neuronsTestPSO.optimise()
    accuraciesForNeuronCounts.append(best_mae_arr[-1])

KeyboardInterrupt: 

In [ ]:
accuraciesForNeuronCounts

[12.77999457152216,
 12.72358284385077,
 12.7507459293109,
 12.128013690751219,
 12.719167595808477,
 12.824716139342142,
 12.408683654740194,
 12.857943071520861]

In [ ]:
accuraciesForAlphas = []

for a in alphaRange:
    network = neuralNet()
    network.add(layer(ActFunc.relu, 32))
    network.add(layer(ActFunc.hb, 64))
    network.add(layer(ActFunc.log, 128))
    network.add(layer(ActFunc.relu, 1))
    alphaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, a, beta, gamma, delta, epsilon, n_iter,prints=True)
    opti_particle, best_mae_arr = alphaTestPSO.optimise()
    accuraciesForAlphas.append(best_mae_arr[-1])

Current best mae: 35.85259317964667
Current best mae: 34.69681357489294
Current best mae: 34.57583386548679
Current best mae: 34.528113056444205
Current best mae: 34.45672812841521
Current best mae: 34.41090745672881
Current best mae: 34.324431491120244
Current best mae: 34.294726777641266
Current best mae: 34.28983005576441
Current best mae: 34.193372938226794
Current best mae: 34.05013041346951
Current best mae: 33.98174222565102
Current best mae: 33.962905969409114
Current best mae: 33.803408176990864
Current best mae: 33.74312486513459
Final best mae 33.74312486513459
Current best mae: 36.02132686084143
Current best mae: 34.86166751582179
Current best mae: 34.71753830103664
Current best mae: 33.145525777035196
Current best mae: 30.782203153340518
Current best mae: 30.385000898299694
Current best mae: 27.099677806233892
Current best mae: 25.250007834601348
Current best mae: 24.461610627611773
Current best mae: 21.39562300399524
Current best mae: 19.69661966269415
Current best mae: 1

In [ ]:
accuraciesForBetas = []

for b in betaRange:
    network = neuralNet()
    network.add(layer(ActFunc.relu, 32))
    network.add(layer(ActFunc.hb, 64))
    network.add(layer(ActFunc.log, 128))
    network.add(layer(ActFunc.relu, 1))
    betaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, b, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = betaTestPSO.optimise()
    accuraciesForBetas.append(best_mae_arr[-1])

In [ ]:
accuraciesForGammas = []

for g in gammaRange:
    network = neuralNet()
    network.add(layer(ActFunc.relu, 32))
    network.add(layer(ActFunc.hb, 64))
    network.add(layer(ActFunc.log, 128))
    network.add(layer(ActFunc.relu, 1))
    gammaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, g, delta, epsilon, n_iter,prints=True)
    opti_particle, best_mae_arr = gammaTestPSO.optimise()
    accuraciesForGammas.append(best_mae_arr[-1])

Current best mae: 35.03732489082014
Current best mae: 34.83481894686642
Current best mae: 34.745755447373725
Current best mae: 34.64531044922929
Current best mae: 34.60332779781707
Current best mae: 32.710327530758406
Current best mae: 28.820975490880095
Current best mae: 23.239086160809002
Current best mae: 23.07334211113402
Current best mae: 21.859346951976192
Current best mae: 16.771328334107093
Current best mae: 14.049072244543881
Current best mae: 13.343404478554394
Current best mae: 12.827673060536709
Current best mae: 12.680568295327712
Final best mae 12.680568295327712
Current best mae: 36.02132686084143
Current best mae: 35.51584165507827
Current best mae: 34.94048970391011
Current best mae: 33.561521441650186
Current best mae: 33.20769068142707
Current best mae: 32.314306604952115
Current best mae: 27.684011543484573
Current best mae: 22.974407913667832
Current best mae: 21.822743236811984
Current best mae: 20.92016318451177
Current best mae: 20.244727407852054
Current best m

In [ ]:
accuraciesForDeltas = []

for d in deltaRange:
    network = neuralNet()
    network.add(layer(ActFunc.relu, 32))
    network.add(layer(ActFunc.hb, 64))
    network.add(layer(ActFunc.log, 128))
    network.add(layer(ActFunc.relu, 1))
    deltaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, d, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = deltaTestPSO.optimise()
    accuraciesForDeltas.append(best_mae_arr[-1])

In [ ]:
accuraciesForEpsilons = []

for e in epsilonRange:
    network = neuralNet()
    network.add(layer(ActFunc.relu, 32))
    network.add(layer(ActFunc.hb, 64))
    network.add(layer(ActFunc.log, 128))
    network.add(layer(ActFunc.relu, 1))
    epsilonTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, e, n_iter,prints=False)
    opti_particle, best_mae_arr = epsilonTestPSO.optimise()
    accuraciesForEpsilons.append(best_mae_arr[-1])

In [ ]:
accuraciesForAlphas

[16.420837013206647,
 12.720444335810935,
 12.718655332856695,
 12.722071306269948,
 12.719791479367064]

In [ ]:
accuraciesForBetas

In [ ]:
accuraciesForGammas

[12.912423059242027,
 12.607260914762906,
 12.718739317776018,
 14.180441401854601,
 13.854707487624232,
 13.506649285547871,
 36.02132686084143,
 13.11766997924925,
 36.02132686084143,
 36.02132686084143]

In [ ]:
accuraciesForDeltas

In [ ]:
accuraciesForEpsilons